### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

today = date.today()
today

datetime.date(2025, 1, 26)

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-01-26
yesterday: 2025-01-24 00:00:00


In [5]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 8)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2025-01-16
yesterday: 2025-01-24


### Restart & Run All Cells

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [9]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Weekly
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### This process affects only already in port stocks. To highlight price changes in percent.

In [12]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [14]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,TVO,2024-11-06,24.90


In [16]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'TVO'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,585,TVO,SETTHSI / SETWB,29.00,31.25,26.50,11.18,2.40,889.47,"25,794.67",18.84,0.43,585,2017-07-23 07:25:46.640728,2023-01-31 03:08:01.449850


In [18]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "TVO" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,TVO,2025-01-24,21.40,21.50,21.10,"185,256",21.20
1,TVO,2025-01-23,21.00,21.20,21.00,"346,263",21.00
2,TVO,2025-01-22,21.00,21.30,21.00,"291,984",21.10
3,TVO,2025-01-21,21.10,21.30,21.10,"238,941",21.20
4,TVO,2025-01-20,21.10,21.30,21.10,"439,007",21.20


In [20]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (yesterday, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2025-01-24' AND name IN ('3BBIF', 'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'TVO', 'WHAIR', 'WHART')
ORDER BY name, date


31

In [22]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (week_ago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2025-01-16' AND name IN ('3BBIF', 'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'TVO', 'WHAIR', 'WHART') 
ORDER BY name


31

In [24]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(31, 3)

In [26]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [28]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [30]:
trend.shape

(31, 4)

In [32]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [34]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
1,AH,13.80,15.00,-8.00%,Worse
19,RCL,25.00,25.75,-2.91%,Worse
17,PTG,7.60,7.80,-2.56%,Worse
5,AWC,3.12,3.18,-1.89%,Worse
23,STA,16.60,16.90,-1.78%,Worse
9,GVREIT,6.55,6.65,-1.50%,Worse
18,PTTGC,22.40,22.60,-0.88%,Worse
14,MCS,6.35,6.35,0.00%,No Change
24,SYNEX,13.20,13.20,0.00%,No Change
20,SCC,156.00,156.00,0.00%,No Change


In [36]:
file_name = 'trend.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [38]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\trend.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\trend.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\trend.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\trend.csv


In [40]:
trend.sort_values(['percent'],ascending=[True]).to_csv(output_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(god_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(icd_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(osd_file, index=False)

### Filter only performance = "Worse"

In [42]:
mask = trend.perform == 'Worse'
trend[mask].shape

(7, 5)

In [44]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,51.61%
No Change,25.81%
Worse,22.58%


In [46]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(246, 8)

In [48]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [50]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(31, 13)

In [52]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
5,AWC,3.18,3.12,-1.89%,Worse,SET50
18,PTTGC,22.60,22.40,-0.88%,Worse,SET50
20,SCC,156.00,156.00,0.00%,No Change,SET50
11,JMART,11.20,11.30,0.89%,Better,SET50
10,IVL,23.20,23.70,2.16%,Better,SET50
12,JMT,14.90,15.40,3.36%,Better,SET50


In [54]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,50.00%
Worse,33.33%
No Change,16.67%


In [56]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
19,RCL,25.75,25.00,-2.91%,Worse,SET100
17,PTG,7.80,7.60,-2.56%,Worse,SET100
23,STA,16.90,16.60,-1.78%,Worse,SET100
16,ORI,3.42,3.42,0.00%,No Change,SET100
6,BCH,14.60,15.00,2.74%,Better,SET100
13,KCE,22.40,23.10,3.13%,Better,SET100
22,SINGER,6.80,7.10,4.41%,Better,SET100


In [58]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,42.86%
Worse,42.86%
No Change,14.29%


In [60]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
1,AH,15.00,13.80,-8.00%,Worse,SET999
9,GVREIT,6.65,6.55,-1.50%,Worse,SET999
2,AIMIRT,10.50,10.50,0.00%,No Change,SET999
4,ASP,2.32,2.32,0.00%,No Change,SET999
14,MCS,6.35,6.35,0.00%,No Change,SET999
21,SENA,2.20,2.20,0.00%,No Change,SET999
24,SYNEX,13.20,13.20,0.00%,No Change,SET999
29,WHAIR,6.45,6.45,0.00%,No Change,SET999
28,TVO,21.30,21.40,0.47%,Better,SET999
3,ASK,8.80,8.85,0.57%,Better,SET999


In [62]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,55.56%
No Change,33.33%
Worse,11.11%


In [64]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [66]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,


In [68]:
const.close()
conmy.close()